# S1W3D1 (Day 15): CNN理论与模型构建

# 🎯 今日目标

1. **理论（面试）**：理解CNN相比于全连接网络（MLP）的两大核心优势。
2. **编码（PyTorch）**：掌握`nn.Conv2d`、`nn.MaxPool2d`和`nn.Flatten`（或`.view`）的用法。
3. **产出（项目）**：从零开始，在 PyTorch 中定义一个完整的 SimpleCNN 模型，并用“假数据”跑通它。

## 🧠 核心理论 (面试考点)

**面试问题 1：为什么处理图像 (如 28x28 的 MNIST) 不直接用全连接网络 (MLP)？**

- 答案1：参数量爆炸（Parameter Explosion）
    - 假设输入是 `28x28` 的灰度图，拉平后是 `784` 个像素。
    - 如果隐藏层有 `500` 个神经元，光是第一层，参数量就是 $784 \times 500 + 500 \text{ (bias)} \approx 39 \text{万}$。
    - 如果输入是 `224x224` 的彩色图 (拉平后 `150,528` 个像素)，参数量将直接**爆炸**，模型根本无法训练。

- 答案2：空间结构丢失 (Loss of Spatial Structure)
    - MLP 将图像“拉平”成一个长向量，完全丢弃了像素之间的空间关系（例如，一个像素和它**旁边**的像素紧密相关，和**远处**的像素关系不大）。

**面试问题 2：CNN (卷积神经网络) 是如何解决这两个问题的？**

  * **答案1：局部感受野 (Local Receptive Fields)**
      * **是什么**：CNN 的一个神经元（在 `nn.Conv2d` 层）只连接到输入图像的一个**局部小区域**（例如 $3 \times 3$ 或 $5 \times 5$ 的“感受野”）。
      * **为什么**：它模拟了人类视觉，先识别局部特征（如边缘、角点），再将它们组合成更复杂的特征。
  * **答案2：参数共享 (Parameter Sharing)**
      * **是什么**：在一个卷积层中，用于扫描整个图像的那个“局部过滤器”（也叫“卷积核”或 Kernel）是**同一个**。
      * **为什么 (好处)**：
        1.  **参数量骤减**：一个 $3 \times 3$ 的卷积核只有 9 个参数，无论图像多大！
        2.  **平移不变性 (Translation Invariance)**：在图像左上角训练好的“边缘检测器”（那个 $3 \times 3$ 的核），同样能识别出右下角的边缘，因为它在共享参数。

## 🛠️ PyTorch 核心模块 (代码实战)

### 模块 1: `nn.Conv2d` (卷积层)

这是CNN的核心，用于提取特征。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 准备一个“假数据”
# 批量大小 B=1, 通道数 C=1 (灰度图), 高 H=28, 宽 W=28
dummy_data = torch.randn(1, 1, 28, 28)
print("输入数据形状:", dummy_data.shape)

# 定义一个兼具曾
# in_channels=1:   输入通道数为1 (灰度图)
# out_channels=16: 输出通道数为16 (卷积核数量)
# kernel_size=3:   卷积核大小为3x3
# stride=1:       步幅为1
# padding=1:      填充为1 (保持输入输出尺寸相同)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)

# 使用卷积层处理“假数据”
output_conv1 = conv1(dummy_data)
print("Conv1 输出形状:", output_conv1.shape)

### 模块 2: `nn.MaxPool2d` (最大池化层)

用于“下采样”，即缩小图像尺寸，同时保留最显著的特征。

In [ ]:
# 定义一个池化层
# kernel_size=2: 池化窗口大小为2x2
# stride=2:      步幅为2 (每次移动2个像素
pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

output_pool1 = pool1(output_conv1)
print(f"Conv1 输出形状: {output_conv1.shape}")
print(f"Pool1 输出形状:, {output_pool1.shape}")
# 预期输出: [1, 16, 14, 14] (H 和 W 都减半了)

### 模块 3: 压平 (`.view` 或 `nn.Flatten`)

在 W2 中，我们使用 `nn.Linear` 必须传入 2D 张量 `(B, Features)`。但现在我们有 4D 张量 `(B, C, H, W)`。我们需要将其“压平”。

In [ ]:
# 方法1：使用 .view() (更灵活，常用于 forward 方法中)
# output_pool1.shape 是 [1, 16, 14, 14]
# B = output_pool1.size(0)  (这里 B=1)
# -1 会自动计算剩余的所有维度 (16 * 14 * 14)
flattened_output_view = output_pool1.view(output_pool1.size(0), -1)
print(f"压平后的形状: {flattened_output_view.shape}")

# 方法2：使用 nn.Flatten() (作为模型的一层)
flatten_layer = nn.Flatten()
flattened_output_layer = flatten_layer(output_pool1)
print(f"使用 nn.Flatten 压平后的形状: {flattened_output_layer.shape}")

## 💻 今日产出：定义 `SimpleCNN` 模型

> 这是今天的**核心任务**。在你的 .ipynb 中创建一个新单元格，定义这个类。

我们将构建一个经典的 CNN 结构：`(Conv -> ReLU -> Pool) -> (Conv -> ReLU -> Pool) -> (Flatten -> Linear -> ReLU -> Linear)`。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # --- 卷积块1 ---
        # 输入: [B, 1, 28, 28]
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3,stride=1, padding=1)
        # 经过 conv1: [B, 16, 28, 28]
        # 经过 ReLU: [B, 16, 28, 28]
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 经过 pool1: [B, 16, 14, 14]

        # --- 卷积块 2 ---
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        # 经过 conv2: [B, 32, 14, 14]
        # 经过 ReLU: [B, 32, 14, 14]
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 经过 pool2: [B, 32, 7, 7]

        # --- 全连接 (分类) 块 ---
        # 我们需要将 [B, 32, 7, 7] 压平为 [B, 32 * 7 * 7] 即 [B, 1568]
        self.fc1 = nn.Linear(in_features=32 * 7 * 7, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=10) # MNIST 是 10 分类

    def forward(self, x):
        # 卷积块1
        x = self.conv1(x)   
        print(f"1. 经过 Conv1: {x.shape}") # 这里会显示 [B, 16, 28, 28]
        x = F.relu(x)
        x = self.pool1(x)
        print(f"2. 经过 Pool1: {x.shape}") # 这里会显示 [B, 16, 14, 14]

        # 卷积块2
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool2(x)

        # 压平
        x = x.view(x.size(0), -1)  # 或者使用 nn
        # 全连接块
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)     # 输出 logits
        return x

## ✅ 单元测试 (Unit Test)

> 马上测试你刚写的模型！确保它能跑通。

In [ ]:
# 1. 实例化模型
model = SimpleCNN()
print("模型结构:")
print(model)

# 2. 创建一个“假数据”批次 (Batch)
# 假设 B=4
dummy_batch = torch.randn(4, 1, 28, 28)

# 3. 运行模型 (前向传播)
try:
    output = model(dummy_batch)
    print("\n--- 单元测试成功 ---")
    print(f"输入形状: {dummy_batch.shape}")
    print(f"输出形状: {output.shape}")
    
    # 检查输出形状是否正确
    assert output.shape == (4, 10)
    print("输出形状正确! (B, num_classes)")
    
except Exception as e:
    print(f"\n--- 单元测试失败 ---")
    print(e)

## 📚 今日总结

今天，我们从理论上理解了 CNN 的**局部感受野**和**参数共享**两大优势，并（更重要的是）在 PyTorch 中使用 `nn.Conv2d` 和 `nn.MaxPool2d` 成功构建了一个完整的 `SimpleCNN` 模型，并验证了它的输入输出形状。

我们已经完成了项目的第一块积木：**Model**。

**明天 (W3D2) 的任务**：准备数据。我们将学习使用 `torchvision.datasets` 和 `transforms` 来加载 *真正* 的 MNIST 数据集，并构建 `DataLoader` 数据管道。

你是否已经完成了今天的全部内容，并成功跑通了单元测试？